In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
dir_path = './data/'

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [3]:
cid = pd.read_excel('aladin_Category_CID_20210927.xls')
cid = cid[['CID', '몰', '카테고리명',  '1Depth', '2Depth']]
cid.columns = ['CID', 'mall', 'category_name',  '1Depth', '2Depth']
cid

,CID,mall,category_name,1Depth,2Depth
0,1230,국내도서,가정/요리/뷰티,가정/요리/뷰티,NaN
1,90452,국내도서,가계부,가정/요리/뷰티,가계부
2,53471,국내도서,건강요리,가정/요리/뷰티,건강요리
3,53489,국내도서,결혼/가족,가정/요리/뷰티,결혼/가족
4,53513,국내도서,가정의례,가정/요리/뷰티,결혼/가족
...,...,...,...,...,...
19581,40697,전자책,임신/출산,Gift,특별한 날
19582,40698,전자책,입학/졸업,Gift,특별한 날
19583,40730,전자책,직접 선물 만들기,Gift,특별한 날
19584,41004,전자책,추석,Gift,특별한 날


In [4]:
cid['CID']

0         1230
1        90452
2        53471
3        53489
4        53513
         ...  
19581    40697
19582    40698
19583    40730
19584    41004
19585    41075
Name: CID, Length: 19586, dtype: int64

In [5]:
import os
import sys
import requests
import json

#키 정의
key = 'ttbsmk9805101528002'

## 1.1. Best seller(Book)

In [6]:
book_list1 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=Bestseller&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list1.append(book_dict)
best_book = pd.DataFrame(book_list1)
best_book = best_book.drop_duplicates()

In [7]:
best_df = pd.merge(best_book, cid, how='left', left_on='CID', right_on='CID')
best_df

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9791168473690,309295168,세이노의 가르침,세이노 (지은이),803200,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
1,9788954691741,314401310,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령, 이서수, 정선임, 함윤이, 현호정 (지은이)",124140,50993,8,6930,7700,2023-04-05,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
2,9791159098444,314773690,던전앤파이터 진각성 아트북 세트 - 전2권,(주)네오플 (지은이),48910,50988,6,76500,85000,2023-04-13,교보문고(단행본),국내도서,화집,예술/대중문화,미술
3,9791160809862,313957212,프린키피아,"아이작 뉴턴 (지은이), 박병철 (옮긴이)",19110,51293,10,59400,66000,2023-04-10,휴머니스트,국내도서,고전물리학,과학,물리학
4,9788970128337,314179276,달리기를 말할 때 내가 하고 싶은 이야기 (리커버 한정판),"무라카미 하루키 (지은이), 임홍빈 (옮긴이)",108925,51373,9,13050,14500,2023-03-30,문학사상,국내도서,외국에세이,에세이,외국에세이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9791130698229,314901463,영혼의 설계자 - 브랜딩을 뛰어넘는 나이키 세계관의 비밀,"그레그 호프먼 (지은이), 이영래 (옮긴이)",1800,1632,10,19800,22000,2023-04-13,다산북스,국내도서,마케팅/브랜드,경제경영,마케팅/세일즈
996,9788934966869,314833700,불의 날개와 어둠의 용 : 하,"투이 T. 서덜랜드 (지은이), 정은규 (그림), 강동혁 (옮긴이)",1720,48877,0,10800,12000,2023-04-11,김영사,국내도서,외국창작동화,어린이,동화/명작/고전
997,9788934942993,314831272,불의 날개와 어둠의 용 : 상,"투이 T. 서덜랜드 (지은이), 정은규 (그림), 강동혁 (옮긴이)",1670,48877,0,10800,12000,2023-04-11,김영사,국내도서,외국창작동화,어린이,동화/명작/고전
998,9791127460662,314795422,평온세대의 위타천들 6,"쿨교신자 (지은이), 아마하라 (원작), 문연주 (옮긴이)",1550,4133,0,4950,5500,2023-04-15,AK(에이케이)커뮤니케이션즈,국내도서,액션 판타지,만화,본격장르만화


## 1.2. Best seller(e-book)

In [8]:
book_list2 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=Bestseller&SearchTarget=eBook" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list2.append(book_dict)
best_e_book = pd.DataFrame(book_list2)
best_e_book = best_e_book.drop_duplicates()

In [9]:
best_e_df = pd.merge(best_e_book, cid, how='left', left_on='CID', right_on='CID')
best_e_df

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9791139114126,314949415,[고화질] [라르고] 콩알탄의 러브 송,스즈마루 민타 (지은이),12220,127587,9,4410,4900,2023-04-13,(주)조은세상,전자책,국외,BL,BL만화
1,9791137333628,305141014,[고화질] 최애의 아이 08,"아카사카 아카 (지은이), 요코야리 멘고 (그림)",14708,40336,10,3150,3500,2022-11-17,대원씨아이/DCW,전자책,레이디스 코믹,만화,순정만화
2,9791170591382,314379780,[고화질] 스킵과 로퍼 08,타카마츠 미사키 (지은이),31460,38563,10,3150,3500,2023-04-03,시프트코믹스,전자책,인간 드라마,만화,인간 드라마
3,9791137338975,314949460,[고화질] 사카모토 데이즈 09,스즈키 유우토 (지은이),9910,38562,10,3150,3500,2023-04-13,대원씨아이/DCW,전자책,코믹/명랑만화,만화,코믹/명랑만화
4,9791138025607,314949318,[고화질] 이세계 삼촌 08,"호톤도신데이루 (지은이), 김민준 (옮긴이)",9110,38798,10,4050,4500,2023-03-16,픽시하우스,전자책,인터넷 연재 카툰,만화,인터넷 연재 카툰
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9791193054291,314776702,갑부 언니로 살기로 마음먹었다 - 당신도 할 수 있다 ! 억대연봉,김의숙(갑부언니) (지은이),360,56618,0,10000,10000,2023-04-07,작가와,전자책,한국에세이,에세이,한국에세이
996,9791169790215,314773269,"먹어주세요, 귀신님! (완전판) 17화",반 (지은이),360,141130,0,500,500,2023-04-10,대원씨아이,전자책,만화,BL,BL연재
997,9791135974809,314827043,[미즈] 됐으니까 솔직하게 느껴 봐 2화 (완결),미야소우 사키 (지은이),350,178786,0,600,600,2023-04-11,넥스큐브,전자책,성인,만화,웹툰
998,9791162368800,314773285,[미즈] 잠자는 척해도 소용없어 50화,토야마 토나리 (지은이),350,178786,0,500,500,2023-04-10,넥스큐브,전자책,성인,만화,웹툰


## 1.3. Best seller(Book + eBook)

In [10]:
best = pd.concat((best_df, best_e_df))
best = best.drop_duplicates()
best.index = [i for i in range(len(best.index))]
best

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9791168473690,309295168,세이노의 가르침,세이노 (지은이),803200,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
1,9788954691741,314401310,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령, 이서수, 정선임, 함윤이, 현호정 (지은이)",124140,50993,8,6930,7700,2023-04-05,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
2,9791159098444,314773690,던전앤파이터 진각성 아트북 세트 - 전2권,(주)네오플 (지은이),48910,50988,6,76500,85000,2023-04-13,교보문고(단행본),국내도서,화집,예술/대중문화,미술
3,9791160809862,313957212,프린키피아,"아이작 뉴턴 (지은이), 박병철 (옮긴이)",19110,51293,10,59400,66000,2023-04-10,휴머니스트,국내도서,고전물리학,과학,물리학
4,9788970128337,314179276,달리기를 말할 때 내가 하고 싶은 이야기 (리커버 한정판),"무라카미 하루키 (지은이), 임홍빈 (옮긴이)",108925,51373,9,13050,14500,2023-03-30,문학사상,국내도서,외국에세이,에세이,외국에세이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9791193054291,314776702,갑부 언니로 살기로 마음먹었다 - 당신도 할 수 있다 ! 억대연봉,김의숙(갑부언니) (지은이),360,56618,0,10000,10000,2023-04-07,작가와,전자책,한국에세이,에세이,한국에세이
1996,9791169790215,314773269,"먹어주세요, 귀신님! (완전판) 17화",반 (지은이),360,141130,0,500,500,2023-04-10,대원씨아이,전자책,만화,BL,BL연재
1997,9791135974809,314827043,[미즈] 됐으니까 솔직하게 느껴 봐 2화 (완결),미야소우 사키 (지은이),350,178786,0,600,600,2023-04-11,넥스큐브,전자책,성인,만화,웹툰
1998,9791162368800,314773285,[미즈] 잠자는 척해도 소용없어 50화,토야마 토나리 (지은이),350,178786,0,500,500,2023-04-10,넥스큐브,전자책,성인,만화,웹툰


In [11]:
best = best[best['customer_review_rank'] != 0]
best.index = [i for i in range(1, len(best.index) + 1)]
best

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9791168473690,309295168,세이노의 가르침,세이노 (지은이),803200,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
2,9788954691741,314401310,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령, 이서수, 정선임, 함윤이, 현호정 (지은이)",124140,50993,8,6930,7700,2023-04-05,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
3,9791159098444,314773690,던전앤파이터 진각성 아트북 세트 - 전2권,(주)네오플 (지은이),48910,50988,6,76500,85000,2023-04-13,교보문고(단행본),국내도서,화집,예술/대중문화,미술
4,9791160809862,313957212,프린키피아,"아이작 뉴턴 (지은이), 박병철 (옮긴이)",19110,51293,10,59400,66000,2023-04-10,휴머니스트,국내도서,고전물리학,과학,물리학
5,9788970128337,314179276,달리기를 말할 때 내가 하고 싶은 이야기 (리커버 한정판),"무라카미 하루키 (지은이), 임홍빈 (옮긴이)",108925,51373,9,13050,14500,2023-03-30,문학사상,국내도서,외국에세이,에세이,외국에세이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,9791168729452,314516054,[세트] 재혼 부부 (총3권/완결),윤소다 (지은이),490,139441,8,9300,9300,2023-04-05,필연매니지먼트,전자책,현대물,로맨스,현대물
1623,9791169387354,314634157,[세트] 별채에는 불이 꺼지지 않는다 (외전 포함) (총4권/완결),정은동 (지은이),480,139441,8,12000,12000,2023-04-07,텐북,전자책,현대물,로맨스,현대물
1624,9791155505502,299825216,"슈퍼 라이브러리 - 공공 도서관을 통해 본 공공 건축, 공공 공간 공공성에 대한 이야기","신승수, 임상진, 최재원 (지은이)",441,57692,6,10000,10000,2022-08-26,사람의무늬,전자책,건축이론/비평/역사,예술/대중문화,건축
1625,9791188063468,314499841,"내일은 후쿠오카 : 나가사키, 벳푸, 유후인 - 2023~2024 최신개정판",온 더 로드 (지은이),410,57575,10,13500,13500,2023-03-10,착한책방,전자책,가이드북,여행,일본여행


## 2.1. blog best(Book)

In [12]:
book_list3 = []

for i in range(1, 5):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=BlogBest&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list3.append(book_dict)
blog_book = pd.DataFrame(book_list3)
blog_book = blog_book.drop_duplicates()

In [13]:
blog_df = pd.merge(blog_book, cid, how='left', left_on='CID', right_on='CID')
blog_df

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9788934981190,313968193,언어의 무게,"파스칼 메르시어 (지은이), 전은경 (옮긴이)",26960,50922,10,19800,22000,2023-04-03,비채,국내도서,독일소설,소설/시/희곡,독일소설
1,9791192107899,285827538,"갈대 속의 영원 - 저항하고 꿈꾸고 연결하는 발명품, 책의 모험","이레네 바예호 (지은이), 이경민 (옮긴이)",13260,51537,10,23400,26000,2023-03-20,반비,국내도서,책읽기,인문학,책읽기/글쓰기
2,9791168126121,314433265,퍼핏 쇼,"M. W. 크레이븐 (지은이), 김해온 (옮긴이)",8130,51062,10,16200,18000,2023-04-06,위즈덤하우스,국내도서,영미 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
3,9788954691741,314401310,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령, 이서수, 정선임, 함윤이, 현호정 (지은이)",124140,50993,8,6930,7700,2023-04-05,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
4,9791156754077,314115340,괴롭힘은 어떻게 뇌를 망가뜨리는가 - 최신 신경과학이 밝히는 괴롭힘의 상처를 치유하는 법,"제니퍼 프레이저 (지은이), 정지호 (옮긴이)",18090,51515,10,23400,26000,2023-04-05,심심,국내도서,심리치료,인문학,심리학/정신분석학
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,9788974832278,517166,"서유견문 - 조선 지식인 유길준, 서양을 번역하다","유길준 (지은이), 허경진 (옮긴이)",1585,118,9,15030,16700,2004-10-01,서해문집,국내도서,조선후기(영조~순종),역사,조선사
138,9791167961143,314844811,섹스로봇과 자살기계 - 인간의 삶과 죽음까지 뒤흔드는 미래 기술의 충격,"제니 클리먼 (지은이), 고호관 (옮긴이)",310,51008,0,16200,18000,2023-04-07,반니,국내도서,미래학,사회과학,미래학
139,9791192512259,314869239,일본 현지 빵 대백과,"타쓰미출판 편집부 (지은이), 수키 (옮긴이)",3980,50854,0,16200,18000,2023-04-17,클,국내도서,일본여행 가이드북,여행,일본여행
140,9791192836003,312516661,서평가의 독서법 - 분열과 고립의 시대의 책읽기,"미치코 가쿠타니 (지은이), 김영선 (옮긴이)",12220,51537,10,17820,19800,2023-03-13,돌베개,국내도서,책읽기,인문학,책읽기/글쓰기


In [14]:
blog_df = blog_df[blog_df['customer_review_rank'] != 0]
blog_df.index = [i for i in range(1, len(blog_df.index) + 1)]
blog_df

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9788934981190,313968193,언어의 무게,"파스칼 메르시어 (지은이), 전은경 (옮긴이)",26960,50922,10,19800,22000,2023-04-03,비채,국내도서,독일소설,소설/시/희곡,독일소설
2,9791192107899,285827538,"갈대 속의 영원 - 저항하고 꿈꾸고 연결하는 발명품, 책의 모험","이레네 바예호 (지은이), 이경민 (옮긴이)",13260,51537,10,23400,26000,2023-03-20,반비,국내도서,책읽기,인문학,책읽기/글쓰기
3,9791168126121,314433265,퍼핏 쇼,"M. W. 크레이븐 (지은이), 김해온 (옮긴이)",8130,51062,10,16200,18000,2023-04-06,위즈덤하우스,국내도서,영미 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
4,9788954691741,314401310,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령, 이서수, 정선임, 함윤이, 현호정 (지은이)",124140,50993,8,6930,7700,2023-04-05,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
5,9791156754077,314115340,괴롭힘은 어떻게 뇌를 망가뜨리는가 - 최신 신경과학이 밝히는 괴롭힘의 상처를 치유하는 법,"제니퍼 프레이저 (지은이), 정지호 (옮긴이)",18090,51515,10,23400,26000,2023-04-05,심심,국내도서,심리치료,인문학,심리학/정신분석학
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,9791156162940,313562421,"오늘은 이렇게 보냈습니다 - 소소하지만 의미 있게, 외롭지 않고 담담하게","무레 요코 (지은이), 손민수 (옮긴이)",1510,51373,10,15120,16800,2023-03-20,리스컴,국내도서,외국에세이,에세이,외국에세이
116,9791168127012,312732931,파쇄,구병모 (지은이),29130,50993,9,11700,13000,2023-03-08,위즈덤하우스,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
117,9788974832278,517166,"서유견문 - 조선 지식인 유길준, 서양을 번역하다","유길준 (지은이), 허경진 (옮긴이)",1585,118,9,15030,16700,2004-10-01,서해문집,국내도서,조선후기(영조~순종),역사,조선사
118,9791192836003,312516661,서평가의 독서법 - 분열과 고립의 시대의 책읽기,"미치코 가쿠타니 (지은이), 김영선 (옮긴이)",12220,51537,10,17820,19800,2023-03-13,돌베개,국내도서,책읽기,인문학,책읽기/글쓰기


## 3.1. Best + Blog

In [15]:
best_blog = pd.concat((best, blog_df))
best_blog = best_blog.drop_duplicates()
best_blog.index = [i for i in range(len(best_blog.index))]
best_blog

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9791168473690,309295168,세이노의 가르침,세이노 (지은이),803200,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
1,9788954691741,314401310,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령, 이서수, 정선임, 함윤이, 현호정 (지은이)",124140,50993,8,6930,7700,2023-04-05,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
2,9791159098444,314773690,던전앤파이터 진각성 아트북 세트 - 전2권,(주)네오플 (지은이),48910,50988,6,76500,85000,2023-04-13,교보문고(단행본),국내도서,화집,예술/대중문화,미술
3,9791160809862,313957212,프린키피아,"아이작 뉴턴 (지은이), 박병철 (옮긴이)",19110,51293,10,59400,66000,2023-04-10,휴머니스트,국내도서,고전물리학,과학,물리학
4,9788970128337,314179276,달리기를 말할 때 내가 하고 싶은 이야기 (리커버 한정판),"무라카미 하루키 (지은이), 임홍빈 (옮긴이)",108925,51373,9,13050,14500,2023-03-30,문학사상,국내도서,외국에세이,에세이,외국에세이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,9791187038931,314832731,일할 자격 - 게으르고 불안정하며 늙고 의지 없는… ‘나쁜 노동자’들이 말하는 노동...,희정 (지은이),1800,51078,10,15300,17000,2023-04-19,갈라파고스,국내도서,노동문제,사회과학,사회문제
1694,9791138477871,313253631,두 번의 작별,"치넨 미키토 (지은이), 민경욱 (옮긴이)",1525,51058,9,14220,15800,2023-03-23,㈜소미미디어,국내도서,일본 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
1695,9791156162940,313562421,"오늘은 이렇게 보냈습니다 - 소소하지만 의미 있게, 외롭지 않고 담담하게","무레 요코 (지은이), 손민수 (옮긴이)",1510,51373,10,15120,16800,2023-03-20,리스컴,국내도서,외국에세이,에세이,외국에세이
1696,9788974832278,517166,"서유견문 - 조선 지식인 유길준, 서양을 번역하다","유길준 (지은이), 허경진 (옮긴이)",1585,118,9,15030,16700,2004-10-01,서해문집,국내도서,조선후기(영조~순종),역사,조선사


In [16]:
best_blog['isbn'].value_counts()

0                6
9791168473690    1
9788925594262    1
9791190538527    1
9788954686556    1
                ..
9791197377143    1
9788997396870    1
9791169990967    1
9791192300566    1
9791191400106    1
Name: isbn, Length: 1693, dtype: int64

In [17]:
best_blog[(best_blog['isbn']=='0') | (best_blog['isbn'] == '9791192500089')]

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
852,0,314672869,[대여 페이백] 당신 엄마가 당신보다 잘하는 게임,박서련 (지은이),5340,57637,9,9100,9100,2023-04-11,민음사,전자책,2000년대 이후,소설/시/희곡,한국소설
967,0,48759842,폭풍의 언덕 (한글판 + 영문판),"에밀리 브론테 (지은이), 김명신 (옮긴이)",14085,40078,9,990,990,2014-11-05,더클래식,전자책,서양근대문학,고전,서양고전문학
1167,0,110853184,6월 민주화대투쟁 (1987년 7월 출간),한국기독교사회문제연구원 (지은이),20933,58331,10,0,0,2020-02-02,민중사,전자책,한국전쟁 이후~현재,역사,한국근현대사
1261,0,199734335,클루지 : 생각의 역사를 뒤집는 기막힌 발견 - 생각의 역사를 뒤집는 기막힌 발견,최호영 (옮긴이),6732,57973,7,8700,9660,2019-07-24,갤리온,전자책,교양 심리학,인문학,심리학/정신분석학
1330,0,250577227,"바뀐 주택임대차보호법 22문22답 - 31년 만의 법개정, 세입자 권리 어떻게 달라...","참여연대, 민주사회를 위한 변호사모임 (지은이)",5308,57916,10,0,0,2020-09-01,참여연대,전자책,생활법률 일반,사회과학,법과 생활
1511,0,33815551,인간실격 (한글판+영문판) - No Longer Human,"다자이 오사무 (지은이), 김소영 (옮긴이)",5030,57639,8,990,990,2013-12-03,더클래식,전자책,1950년대 이전,소설/시/희곡,일본소설


## 4.1 New Special(Book)

In [18]:
book_list4 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=ItemNewSpecial&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list4.append(book_dict)
new_book = pd.DataFrame(book_list4)
new_book = new_book.drop_duplicates()

In [19]:
new_book = pd.merge(new_book, cid, how='left', left_on='CID', right_on='CID')
new_book.index = [i for i in range(1, len(new_book.index) + 1)]
new_book

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9791169811347,315179679,잃어버린 얼굴,"올가 토카르추크 (지은이), 요안나 콘세이요 (그림), 이지원 (옮긴이)",0,51036,0,16200,18000,2023-04-28,사계절,국내도서,동유럽소설,소설/시/희곡,세계의 소설
2,9791168126152,315173460,"악동 김블루의 친절한 과학 3 - 지구, 전류와 전압, 대기와 해양","악동 김블루 (원작), 조영선 (글), 오차(이영아) (그림), 샌드박스 네트워크,...",0,48867,0,13500,15000,2023-04-19,위즈덤하우스,국내도서,과학 일반,어린이,과학/수학/컴퓨터
3,9788954699457,315160552,새벽 2시의 코인 세탁소,박현주 (지은이),0,51065,0,13500,15000,2023-04-17,엘릭시르,국내도서,한국 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
4,9791191783063,315159347,인어의 비탄 * 마술사,"다니자키 준이치로 (지은이), 박성민 (옮긴이)",0,50996,0,10800,12000,2023-04-15,시와서,국내도서,1950년대 이전 일본소설,소설/시/희곡,일본소설
5,9791169831703,315158902,뉴 레트로 드로잉 테크닉 - 복고 스타일까지 완벽하게 마무리하기,"DenQ, 야베 사와코, 나카무라 교코, nika, nui, empire hotel...",0,2564,0,19800,22000,2023-04-28,므큐,국내도서,만화작법/일러스트,만화,만화그리기와 읽기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,,312178695,[세트] 휴머니스트 세계문학 시즌 4 : 결정적 한순간 - 전5권,"어니스트 헤밍웨이, 부스 타킹턴, 그라치아 델레다, 세르브 언털, 헤르만 헤세 (지...",170,50919,10,64800,72000,2023-03-06,휴머니스트,국내도서,영미소설,소설/시/희곡,영미소설
997,9791197694622,312178327,로컬 브랜드 리뷰 2023 - 로컬이 강한 도시와 동네,"모종린, 김보민, 박예솔 (지은이)",3600,1632,10,17100,19000,2023-02-27,포틀랜드스쿨,국내도서,마케팅/브랜드,경제경영,마케팅/세일즈
998,9791157062812,312175479,소설의 쓸모 - 우리에게 필요한 이야기들,박산호 (지은이),2170,51371,10,12600,14000,2023-02-28,ㅁ(미음),국내도서,한국에세이,에세이,한국에세이
999,9791188754779,312174758,한국형 탑다운 투자 전략 - 이베스트 리서치의 주식 투자 레벨 업 프로젝트,"윤지호, 신중호, 최광혁, 정다운, 최진영 (지은이)",12310,174,9,24300,27000,2023-02-28,에프엔미디어,국내도서,주식/펀드,경제경영,재테크/투자


In [20]:
new_book = new_book[new_book['customer_review_rank'] != 0]
new_book.index = [i for i in range(1, len(new_book.index) + 1)]
new_book

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9791165347161,315155324,나의 돈 많은 고등학교 친구 - 슈퍼리치와의 대화에서 찾아낸 부자의 길,송희구 (지은이),0,70216,10,16200,18000,2023-04-28,서삼독,국내도서,성공학,자기계발,성공
2,9788901269962,314998304,미셸 오바마 자기만의 빛 - 어둠의 시간을 밝히는 인생의 도구들,"미셸 오바마 (지은이), 이다희 (옮긴이)",820,51373,10,18000,20000,2023-04-17,웅진지식하우스,국내도서,외국에세이,에세이,외국에세이
3,9791198125613,314993820,우리의 미래를 결정할 과학 4.0 - 인공지능(AI)에서 아르테미스 프로젝트까지,박재용 (지은이),60,51002,10,17550,19500,2023-04-25,북루덴스,국내도서,기초과학/교양과학,과학,기초과학/교양과학
4,9791191593105,314988457,나는 노동자다 - 슬기로운 노동자 생활,민플러스 교육원 (지은이),530,51129,10,14400,16000,2023-04-15,민플러스,국내도서,노동운동,사회과학,사회운동
5,9788958079545,314978437,백 일의 밤 백 편의 시 - 일상을 충만하게 채우는 시의 언어들,이영주 (지은이),1350,51167,10,15120,16800,2023-04-14,뜨인돌,국내도서,한국시,소설/시/희곡,시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,,312178695,[세트] 휴머니스트 세계문학 시즌 4 : 결정적 한순간 - 전5권,"어니스트 헤밍웨이, 부스 타킹턴, 그라치아 델레다, 세르브 언털, 헤르만 헤세 (지...",170,50919,10,64800,72000,2023-03-06,휴머니스트,국내도서,영미소설,소설/시/희곡,영미소설
636,9791197694622,312178327,로컬 브랜드 리뷰 2023 - 로컬이 강한 도시와 동네,"모종린, 김보민, 박예솔 (지은이)",3600,1632,10,17100,19000,2023-02-27,포틀랜드스쿨,국내도서,마케팅/브랜드,경제경영,마케팅/세일즈
637,9791157062812,312175479,소설의 쓸모 - 우리에게 필요한 이야기들,박산호 (지은이),2170,51371,10,12600,14000,2023-02-28,ㅁ(미음),국내도서,한국에세이,에세이,한국에세이
638,9791188754779,312174758,한국형 탑다운 투자 전략 - 이베스트 리서치의 주식 투자 레벨 업 프로젝트,"윤지호, 신중호, 최광혁, 정다운, 최진영 (지은이)",12310,174,9,24300,27000,2023-02-28,에프엔미디어,국내도서,주식/펀드,경제경영,재테크/투자


## 4.2 New Special(e-book)

In [21]:
book_list5 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=ItemNewSpecial&SearchTarget=eBook" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list5.append(book_dict)
new_ebook = pd.DataFrame(book_list5)
new_ebook = new_ebook.drop_duplicates()

In [22]:
new_ebook = pd.merge(new_ebook, cid, how='left', left_on='CID', right_on='CID')
new_ebook.index = [i for i in range(1, len(new_ebook.index) + 1)]
new_ebook

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9791168129078,315053476,퍼핏 쇼,"M. W. 크레이븐 (지은이), 김해온 (옮긴이)",200,57670,10,12600,12600,2023-04-14,위즈덤하우스,전자책,영미,소설/시/희곡,추리/미스터리소설
2,9791168340961,315053469,저주토끼 - 개정판,정보라 (지은이),150,89505,0,12500,12500,2023-04-13,래빗홀,전자책,한국,소설/시/희곡,과학소설
3,9788954691956,315011238,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령 (지은이)",6450,57637,8,5400,5400,2023-04-14,문학동네,전자책,2000년대 이후,소설/시/희곡,한국소설
4,9791168340916,314930482,내면소통 - 삶의 변화를 가져오는 마음근력 훈련,김주환 (지은이),2580,57979,10,26000,26000,2023-04-15,인플루엔셜(주),전자책,뇌과학/인지심리학,인문학,심리학/정신분석학
5,9791189318420,314925573,한국형 가치투자 - 이론과 실전을 모두 담아 새로 쓴,"최준철, 김민국 (지은이)",2210,39740,9,12150,13500,2023-03-30,이콘,전자책,주식/펀드,경제경영,재테크/투자
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,9791155815809,309336189,1일 1클래식 1포옹 - 하루를 껴안는 음악의 힘,"클레먼시 버턴힐 (지은이), 이석호 (옮긴이)",420,57645,10,12510,13900,2023-01-25,윌북,전자책,서양음악(클래식),예술/대중문화,음악
183,9788960519664,309203026,적당히 느슨하게 조금씩 행복해지는 습관 - 우울증에 괴로워해 본 정신과 의사의 나를...,"바쿠@정신과의 (지은이), 김윤경 (옮긴이)",715,74497,9,11500,11500,2023-01-24,부키,전자책,마음 다스리기,자기계발,힐링
184,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,"야오야오 (지은이), 김진아 (옮긴이)",175,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계
185,9791155815816,308819466,옥스퍼드 오늘의 단어책 - 1일 1단어 1기쁨,"수지 덴트 (지은이), 고정아 (옮긴이)",1700,57996,9,12510,13900,2023-01-30,윌북,전자책,언어학,인문학,기호학/언어학


In [23]:
new_ebook = new_ebook[new_ebook['customer_review_rank'] != 0]
new_ebook.index = [i for i in range(1, len(new_ebook.index) + 1)]
new_ebook

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9791168129078,315053476,퍼핏 쇼,"M. W. 크레이븐 (지은이), 김해온 (옮긴이)",200,57670,10,12600,12600,2023-04-14,위즈덤하우스,전자책,영미,소설/시/희곡,추리/미스터리소설
2,9788954691956,315011238,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령 (지은이)",6450,57637,8,5400,5400,2023-04-14,문학동네,전자책,2000년대 이후,소설/시/희곡,한국소설
3,9791168340916,314930482,내면소통 - 삶의 변화를 가져오는 마음근력 훈련,김주환 (지은이),2580,57979,10,26000,26000,2023-04-15,인플루엔셜(주),전자책,뇌과학/인지심리학,인문학,심리학/정신분석학
4,9791189318420,314925573,한국형 가치투자 - 이론과 실전을 모두 담아 새로 쓴,"최준철, 김민국 (지은이)",2210,39740,9,12150,13500,2023-03-30,이콘,전자책,주식/펀드,경제경영,재테크/투자
5,9791187444893,314925558,부의 추월차선 위대한 탈출 - 경제적 자유를 앞당기는 120가지 원리와 전략,"엠제이 드마코 (지은이), 이영래 (옮긴이)",400,40063,10,13860,15400,2023-04-20,토트,전자책,재테크/투자 일반,경제경영,재테크/투자
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,9791155815809,309336189,1일 1클래식 1포옹 - 하루를 껴안는 음악의 힘,"클레먼시 버턴힐 (지은이), 이석호 (옮긴이)",420,57645,10,12510,13900,2023-01-25,윌북,전자책,서양음악(클래식),예술/대중문화,음악
178,9788960519664,309203026,적당히 느슨하게 조금씩 행복해지는 습관 - 우울증에 괴로워해 본 정신과 의사의 나를...,"바쿠@정신과의 (지은이), 김윤경 (옮긴이)",715,74497,9,11500,11500,2023-01-24,부키,전자책,마음 다스리기,자기계발,힐링
179,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,"야오야오 (지은이), 김진아 (옮긴이)",175,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계
180,9791155815816,308819466,옥스퍼드 오늘의 단어책 - 1일 1단어 1기쁨,"수지 덴트 (지은이), 고정아 (옮긴이)",1700,57996,9,12510,13900,2023-01-30,윌북,전자책,언어학,인문학,기호학/언어학


## 4.3 New Special(Book + ebook)

In [24]:
new_df = pd.concat((new_book, new_ebook))
new_df = new_df.drop_duplicates()
new_df.index = [i for i in range(len(new_df.index))]
new_df

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9791165347161,315155324,나의 돈 많은 고등학교 친구 - 슈퍼리치와의 대화에서 찾아낸 부자의 길,송희구 (지은이),0,70216,10,16200,18000,2023-04-28,서삼독,국내도서,성공학,자기계발,성공
1,9788901269962,314998304,미셸 오바마 자기만의 빛 - 어둠의 시간을 밝히는 인생의 도구들,"미셸 오바마 (지은이), 이다희 (옮긴이)",820,51373,10,18000,20000,2023-04-17,웅진지식하우스,국내도서,외국에세이,에세이,외국에세이
2,9791198125613,314993820,우리의 미래를 결정할 과학 4.0 - 인공지능(AI)에서 아르테미스 프로젝트까지,박재용 (지은이),60,51002,10,17550,19500,2023-04-25,북루덴스,국내도서,기초과학/교양과학,과학,기초과학/교양과학
3,9791191593105,314988457,나는 노동자다 - 슬기로운 노동자 생활,민플러스 교육원 (지은이),530,51129,10,14400,16000,2023-04-15,민플러스,국내도서,노동운동,사회과학,사회운동
4,9788958079545,314978437,백 일의 밤 백 편의 시 - 일상을 충만하게 채우는 시의 언어들,이영주 (지은이),1350,51167,10,15120,16800,2023-04-14,뜨인돌,국내도서,한국시,소설/시/희곡,시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,9791155815809,309336189,1일 1클래식 1포옹 - 하루를 껴안는 음악의 힘,"클레먼시 버턴힐 (지은이), 이석호 (옮긴이)",420,57645,10,12510,13900,2023-01-25,윌북,전자책,서양음악(클래식),예술/대중문화,음악
816,9788960519664,309203026,적당히 느슨하게 조금씩 행복해지는 습관 - 우울증에 괴로워해 본 정신과 의사의 나를...,"바쿠@정신과의 (지은이), 김윤경 (옮긴이)",715,74497,9,11500,11500,2023-01-24,부키,전자책,마음 다스리기,자기계발,힐링
817,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,"야오야오 (지은이), 김진아 (옮긴이)",175,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계
818,9791155815816,308819466,옥스퍼드 오늘의 단어책 - 1일 1단어 1기쁨,"수지 덴트 (지은이), 고정아 (옮긴이)",1700,57996,9,12510,13900,2023-01-30,윌북,전자책,언어학,인문학,기호학/언어학


## 5.1 Best + Blog + New Special

In [25]:
len(best_blog), len(new_df)

(1698, 820)

In [26]:
best_blog_new = pd.concat((best_blog, new_df))
best_blog_new = best_blog_new.drop_duplicates()
best_blog_new.index = [i for i in range(len(best_blog_new.index))]
best_blog_new

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9791168473690,309295168,세이노의 가르침,세이노 (지은이),803200,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
1,9788954691741,314401310,2023 제14회 젊은작가상 수상작품집,"이미상, 김멜라, 성혜령, 이서수, 정선임, 함윤이, 현호정 (지은이)",124140,50993,8,6930,7700,2023-04-05,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
2,9791159098444,314773690,던전앤파이터 진각성 아트북 세트 - 전2권,(주)네오플 (지은이),48910,50988,6,76500,85000,2023-04-13,교보문고(단행본),국내도서,화집,예술/대중문화,미술
3,9791160809862,313957212,프린키피아,"아이작 뉴턴 (지은이), 박병철 (옮긴이)",19110,51293,10,59400,66000,2023-04-10,휴머니스트,국내도서,고전물리학,과학,물리학
4,9788970128337,314179276,달리기를 말할 때 내가 하고 싶은 이야기 (리커버 한정판),"무라카미 하루키 (지은이), 임홍빈 (옮긴이)",108925,51373,9,13050,14500,2023-03-30,문학사상,국내도서,외국에세이,에세이,외국에세이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,9791155815809,309336189,1일 1클래식 1포옹 - 하루를 껴안는 음악의 힘,"클레먼시 버턴힐 (지은이), 이석호 (옮긴이)",420,57645,10,12510,13900,2023-01-25,윌북,전자책,서양음악(클래식),예술/대중문화,음악
2233,9788960519664,309203026,적당히 느슨하게 조금씩 행복해지는 습관 - 우울증에 괴로워해 본 정신과 의사의 나를...,"바쿠@정신과의 (지은이), 김윤경 (옮긴이)",715,74497,9,11500,11500,2023-01-24,부키,전자책,마음 다스리기,자기계발,힐링
2234,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,"야오야오 (지은이), 김진아 (옮긴이)",175,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계
2235,9791155815816,308819466,옥스퍼드 오늘의 단어책 - 1일 1단어 1기쁨,"수지 덴트 (지은이), 고정아 (옮긴이)",1700,57996,9,12510,13900,2023-01-30,윌북,전자책,언어학,인문학,기호학/언어학


In [27]:
best_blog_new['isbn'].value_counts()

0                6
9791168473690    1
9791163430506    1
9791193001011    1
9791140401802    1
                ..
9791169473163    1
9791168809109    1
9791192300528    1
9791160560015    1
9791164840601    1
Name: isbn, Length: 2232, dtype: int64

In [28]:
best_blog_new = best_blog_new.drop(best_blog_new[(best_blog_new['isbn'] == '0') | (best_blog_new['isbn'] == '') | (best_blog_new['isbn'] == '9791192500089')].index) \
                .reset_index(drop=True)
best_blog_new['isbn'].value_counts()

9791168473690    1
9791136828309    1
9791193001011    1
9791140401802    1
9791168827530    1
                ..
9791168809109    1
9791192300528    1
9791160560015    1
9788927712459    1
9791164840601    1
Name: isbn, Length: 2230, dtype: int64

In [29]:
best_blog_new.to_excel(dir_path + "aladin_20230330.xlsx", index = False, header = True)

In [30]:
best_blog_new[best_blog_new['isbn'] == '9788962630619']

,isbn,itemid,title,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
579,9788962630619,14461351,침묵의 봄 - 개정판,"레이첼 카슨 (지은이), 김은령 (옮긴이), 홍욱희 (감수)",50109,51101,9,16200,18000,2011-12-30,에코리브르,국내도서,환경학 일반,사회과학,환경/생태문제
